#### 导入必要的包

In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### 处理数据集

In [2]:
## 读取数据集
data = pd.read_csv("./dataset/credit-a.csv",header=None)  #声明数据集不带有表头信息
data.info # 查看数据集的详细信息

<bound method DataFrame.info of      0      1       2   3   4   5   6     7   8   9   10  11  12   13     14  \
0     0  30.83   0.000   0   0   9   0  1.25   0   0   1   1   0  202    0.0   
1     1  58.67   4.460   0   0   8   1  3.04   0   0   6   1   0   43  560.0   
2     1  24.50   0.500   0   0   8   1  1.50   0   1   0   1   0  280  824.0   
3     0  27.83   1.540   0   0   9   0  3.75   0   0   5   0   0  100    3.0   
4     0  20.17   5.625   0   0   9   0  1.71   0   1   0   1   2  120    0.0   
..   ..    ...     ...  ..  ..  ..  ..   ...  ..  ..  ..  ..  ..  ...    ...   
648   0  21.08  10.085   1   1  11   1  1.25   1   1   0   1   0  260    0.0   
649   1  22.67   0.750   0   0   0   0  2.00   1   0   2   0   0  200  394.0   
650   1  25.25  13.500   1   1  13   7  2.00   1   0   1   0   0  200    1.0   
651   0  17.92   0.205   0   0  12   0  0.04   1   1   0   1   0  280  750.0   
652   0  35.00   3.375   0   0   0   1  8.29   1   1   0   0   0    0    0.0   

     15

In [3]:
X = data.iloc[:,:-1] #取到前十五列数据

In [4]:
Y = data.iloc[:,-1] #渠道最后一列数据

In [5]:
# 根据实际任务以及逻辑回归的需求，将Y输出中的-1改为0 以满足sigmoid函数
Y = Y.replace(-1,0)

In [6]:
# 将数据全部转化为Tensor
X = torch.from_numpy(X.values).type(torch.float32)
Y = torch.from_numpy(Y.values.reshape(-1,1)).type(torch.float32) #转化的过程中顺带指定数据类型

#### 逻辑回归主要逻辑

###### 构建神经网络模型

In [7]:
from torch import nn #导入逻辑回归包含于的神经网络工具箱

In [8]:
model = nn.Sequential( #快速创建神经网络层模型的工具
    nn.Linear(15,1), #创建第一层线性层 输入为15个神经元 输出为1个神经元
    nn.Sigmoid() #创建第二层激活层 所有的层顺序链接
) #将多个层按照顺序连接在一起，快速创建模型

In [9]:
model # 打印model信息

Sequential(
  (0): Linear(in_features=15, out_features=1, bias=True)
  (1): Sigmoid()
)

In [10]:
# 初始化损失函数 这里使用二元交叉熵损失函数 所对应的标签是 0 1
loss_fn = nn.BCELoss()

In [11]:
opt     = torch.optim.Adam(model.parameters(),lr = 0.0001) #使用Adam算法来优化训练过程

###### 使用mini-batch方法对模型进行训练

In [12]:
batch_size      = 16 #定义每一批数据的量
number_of_batch = 653//16

In [13]:
epoches = 1000
for epoch in range(epoches): #训练周期数
    for i in range(number_of_batch): #训练所需要的batch数
        # 计算数据集的切分点
        start = i*batch_size
        end   = start + batch_size
        # 得到mini-batch所需要的数据
        x = X[start:end]
        y = Y[start:end]
        # 套用模型进行预测
        y_pred = model(x)
        # 计算损失 注意第一个参数是 y_pred 第二个参数是y
        loss   = loss_fn(y_pred,y)
        # 梯度清零 为反向传播进行准备
        opt.zero_grad()
        # 进行反向传播
        loss.backward()
        # 直接对参数进行自动优化
        opt.step()

In [14]:
# 查看模型参数
model.state_dict() # 参数的意思就是 sigmoid(w1*x1+w2*x2 + ...... + b)

OrderedDict([('0.weight',
              tensor([[-8.1358e-03, -6.3207e-03, -5.6089e-03,  3.0419e-01,  1.2003e-01,
                       -1.7914e-02,  1.8645e-01, -1.6885e-01,  1.8278e+00,  4.8902e-01,
                       -1.7084e-01,  4.7547e-02, -6.6742e-03,  1.1200e-03, -3.3804e-04]])),
             ('0.bias', tensor([-0.1223]))])

###### 得到结果并测试/预测

In [15]:
model(X).data.numpy() # 根据训练所得到的模型，传入实际的参数，查看预测值

array([[3.57445538e-01],
       [1.07644625e-01],
       [5.18347621e-01],
       [1.36529550e-01],
       [4.81598675e-01],
       [5.03784239e-01],
       [1.26603072e-05],
       [4.47031945e-01],
       [5.03858805e-01],
       [3.47496480e-01],
       [8.82320344e-01],
       [4.73268062e-01],
       [4.56309617e-01],
       [7.10680068e-01],
       [6.52295649e-02],
       [1.92109942e-01],
       [3.44272226e-01],
       [6.92592859e-02],
       [5.70436895e-01],
       [2.11396292e-01],
       [2.17413642e-02],
       [6.09067738e-01],
       [3.70503329e-02],
       [2.90854454e-01],
       [2.67973959e-01],
       [3.81617934e-01],
       [6.97281957e-02],
       [3.45092313e-03],
       [1.00617431e-01],
       [5.24634449e-03],
       [1.06967427e-01],
       [4.31680381e-02],
       [3.36150452e-03],
       [9.14669633e-02],
       [1.69344306e-01],
       [9.88125801e-02],
       [1.09024301e-01],
       [3.51892292e-01],
       [4.68362778e-01],
       [2.12737098e-02],


###### 规范化输出结果

In [16]:
(model(X).data.numpy()>0.5).astype(np.int) #一般认为输出值大于0.5 结果规范化为1 反之规范化为0

C:\Users\20613\AppData\Local\Temp/ipykernel_10868/260771198.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (model(X).data.numpy()>0.5).astype(np.int) #一般认为输出值大于0.5 结果规范化为1 反之规范化为0


array([[0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
    

###### 计算正确率

In [17]:
((model(X).data.numpy()>0.5).astype(np.int)==Y.numpy()).mean() #计算原则 ：输出结果与标签逐个比较 计算相符item占总数的比例

C:\Users\20613\AppData\Local\Temp/ipykernel_10868/3562330135.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ((model(X).data.numpy()>0.5).astype(np.int)==Y.numpy()).mean() #计算原则 ：输出结果与标签逐个比较 计算相符item占总数的比例


0.8621745788667687